# Step 2: Data Collection & Initial EDA of Candidate Datasets
## Dataset 2: PaperDoll/Chictopia

### MLE Capstone: Outfit Recommender - Spring 2021
### By: Bazeley, Mikiko 
### GH: [@mmbazel](https://github.com/MMBazel)  

In this notebook, I'll be exploring one of three datasets (DeepFashion, Paperdoll, and iMaterialist). 

Specifically in this notebook we'll: 

☑️ Load the data

☑️ Explore the dimensions of the dataset

☑️ Understand what categories are being represented

☑️ Explore samples of the data (the meta data dictionary with catgories & attributes labels, the Train 🚂 file, and finally the images 📸 themselves)

☑️ Understand distributions of categories, attributes

<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

## Explanation of the data, according to the dataset page here: 
https://github.com/kyamagu/paperdoll/tree/master/data/chictopia

### Files


### ⚠️📝 Notes (About the Notebooks) ⚠️📝 

My guiding principles:
* ➡️ Be overly communicative = While that leads to verbose commenting, I hope that means I catch a bunch of questions early)  
* ➡️ Human-readable over witty-optimization = For the most part I try to make everything I'm doing obvious
* ➡️ Write as much code as needed, and no more = There's a time and place for error-catching & object-oriented code & there are ways to make the notebook reproducible. That's not quite the goal for this notebook (or any of the other notebooks in the early stages of the project) and my goal was to write just the code needed to get this step done.  

<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style='background :red' > Step 1: Proper set-up & installation of necessary libraries & packages </span> 

1. Ensure you're using the right flavor of commands and that you have sqlite3 available to you. It's easy enough to pull up the terminal or whatever shell version youre using to check. 


In [ ]:
#!pip3 install lmdb
#!pip3 install ipython-sql

In [53]:
%load_ext sql

In [54]:
##################### [TODO] SETUP #####################
# [TODO] Import any utilities functions


import json
import os
import sys
from sqlalchemy import create_engine

import io
import lmdb
import sqlite3
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

print('Packages Imported')

modules = dir()

print(modules)
#print(os.environ)

# [TODO] Package install/load

Packages Imported
['Image', 'In', 'Out', '_', '_15', '_16', '_17', '_18', '_19', '_27', '_30', '_31', '_37', '_38', '_45', '_46', '_47', '_48', '_49', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_exit_code', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i5', '_i50', '_i51', '_i52', '_i53', '_i54', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'color_taggings_sample', 'create_engine', 'db', 'exit', 'garments_sample', 'get_ipython', 'io', 'json', 'lmdb', 'modules', 'np', 'os', 'pd', 'plt', 'query', 'quit', 'schema_list', 'sqlite3', 'stores_sample', 'sys', 't_name', 'table_list_dict', 'table_names_list'

In [3]:
# Confirm all the right libraries are present
# This is an important step because there's a good chance
# that for some pckaes where you use pip or pip3 install
# they could download to the wrong directory if you're not
# using the right pip executable

!conda list

# packages in environment at /Users/mikikobazeley/opt/anaconda3/envs/SPRINGBOARD_MLE_CAPSTONE_ENV:
#
# Name                    Version                   Build  Channel
_py-xgboost-mutex         2.0                       cpu_0  
_pytorch_select           0.1                       cpu_0    anaconda
_tflow_select             2.3.0                       mkl  
absl-py                   0.11.0             pyhd3eb1b0_1  
appnope                   0.1.2           py37hecd8cb5_1001  
argon2-cffi               20.1.0           py37haf1e3a3_1    anaconda
astor                     0.8.1            py37hecd8cb5_0  
async_generator           1.10             py37h28b3542_0    anaconda
attrs                     20.3.0             pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
bleach                    3.3.0              pyhd3eb1b0_0  
blis                      0.7.4                    pypi_0    pypi
brotlipy      

In [4]:
# Confirm path of working directory
!pwd 

/Users/mikikobazeley/Github/personal/MMBazel/Wardrobe-Recommender/notebooks/Step2_EDA


<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style='background :red' > Step 2: Download the Chictopia Datasets</span> 

⚠️ Best way to not mess this process up is to follow the insteructions here exactly: https://github.com/kyamagu/paperdoll/tree/master/data/chictopia

Make sure you have installed: 
➡️ wget (I used homebrew to install & manage so it's globally available on my mac as opposed to just the conda env => !brew install wget)

## <span style='background :orange' > Clone the Github Repo </span> 

In [5]:
# Use git to clone PaperDolls repository. Note Git LFS is used to host metadata file.
# Then navigate to data/chictopia dir.
#!git clone https://github.com/kyamagu/paperdoll /Volumes/MiniGator/Projects/Datasets/Paperdoll

In [6]:
# Confirm zipfile has been downloaded to the repo
# Replace path with the path-to-dowloaded-dataset
!ls -a -F /Volumes/MiniGator/Projects/Datasets/Paperdoll

./              .git/           .gitignore*     README.md*
../             .gitattributes* LICENSE.txt*    data/


In [7]:
!ls -a -F /Volumes/MiniGator/Projects/Datasets/Paperdoll/data

./          ../         .gitignore* chictopia/


In [8]:
!ls -a -F /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia

./                  ._chictopia.sql.gz* chictopia.sql.gz*   photos/
../                 README.md*          chictopia.sqlite3*  photos.lmdb.tar*


In [9]:
#Uncomment first line to install. 
#!brew install wget

## <span style='background :orange' > Download the image data from remote </span> 

In [10]:
# If the command doesn't work as a magic command in Jupyter; 
# run it in the vanilla terminal. 
# Download the photo data (~40GB)

#!wget https://s3-ap-northeast-1.amazonaws.com/kyamagu-public/chictopia2/photos.lmdb.tar -P /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia

In [12]:
# Check again because I'm paranoid
!ls /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/photos.lmdb.tar

/Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/photos.lmdb.tar


## <span style='background :orange' > Unzip photos file </span> 

In [ ]:
# Create a directory to make sure the photos get unzipped to a specific place
#!mkdir /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/photos/

In [ ]:
# Extract the LMDB content
# LMDB stores raw PNG binary. The key is photo id and the value is binary. 
# See the notebook demo for how to open.
# Consider using pv for tracking
# https://stackoverflow.com/questions/19372373/how-to-add-progress-bar-to-a-somearchive-tar-xz-extract
# Also: https://stackoverflow.com/questions/13707429/decompress-gzip-file-to-specific-directory

#!tar -xf /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/photos.lmdb.tar -C /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/photos/

## <span style='background :orange' > Unzip SQL file to create db, tables and load SQLite3 Chictopia DB</span> 

In [ ]:
# First reconstruct the metadata SQLite3 file. 
# The following command will reconstruct the SQLite3 database file.
#!gunzip -c /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/chictopia.sql.gz | sqlite3 /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/chictopia.sqlite3

In [13]:
# Check that we see a sqlite3 file
!ls -F -a /Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/

./                  ._chictopia.sql.gz* chictopia.sql.gz*   photos/
../                 README.md*          chictopia.sqlite3*  photos.lmdb.tar*


<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style='background :red' > Step 3: Use SQL to start initial exploration 🔬  </span> 

We want to understand:

☑️ What tables exist

☑️ The columns & data types for each 

☑️ Entites & relationships being captured



<span style='background :yellow' > 💡 We'll connect using the sqlite3 package & query with pandas instead of setting up the ful blown SQLAlchemy connection for the EDA. </span> 

See https://bit.ly/3r8hUC4 for the example usage notebook included with the dataset's repo.

In [14]:
#Connecting
db = sqlite3.connect('file:/Volumes/MiniGator/Projects/Datasets/Paperdoll/data/chictopia/chictopia.sqlite3?mode=ro', uri=True)

In [61]:
%%sql sqlite:////Volumes//MiniGator//Projects//Datasets//Paperdoll//data//chictopia//chictopia.sqlite3
    
%%sql select * from 'users' 

(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql select * from 'users']
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [15]:
# Gettin schema
schema_list = pd.read_sql("""SELECT * 
FROM sqlite_schema
""",con=db)

schema_list.sample(n=10)

,type,name,tbl_name,rootpage,sql
16,index,index_users_on_street_id,users,392559,"CREATE INDEX ""index_users_on_street_id"" ON ""us..."
48,index,index_friendships_on_user_id_and_to_id,friendships,512678,"CREATE UNIQUE INDEX ""index_friendships_on_user..."
35,index,index_users_on_status,users,477563,"CREATE INDEX ""index_users_on_status"" ON ""users..."
6,table,tags,tags,83538,"CREATE TABLE ""tags"" (""id"" INTEGER PRIMARY KEY ..."
15,table,photos,photos,326399,"CREATE TABLE ""photos"" (""id"" INTEGER PRIMARY KE..."
23,index,index_garments_on_color_id,garments,406651,"CREATE INDEX ""index_garments_on_color_id"" ON ""..."
53,index,index_photos_on_status,photos,553463,"CREATE INDEX ""index_photos_on_status"" ON ""phot..."
0,table,schema_migrations,schema_migrations,2,"CREATE TABLE ""schema_migrations"" (""version"" va..."
39,index,index_users_on_updated_at,users,497368,"CREATE INDEX ""index_users_on_updated_at"" ON ""u..."
3,table,posts,posts,6519,"CREATE TABLE ""posts"" (""id"" INTEGER PRIMARY KEY..."


In [16]:
# Preview tables
schema_list.loc[schema_list['type'] == 'table']

,type,name,tbl_name,rootpage,sql
0,table,schema_migrations,schema_migrations,2,"CREATE TABLE ""schema_migrations"" (""version"" va..."
1,table,users,users,3,"CREATE TABLE ""users"" (""id"" INTEGER PRIMARY KEY..."
2,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
3,table,posts,posts,6519,"CREATE TABLE ""posts"" (""id"" INTEGER PRIMARY KEY..."
4,table,streets,streets,73296,"CREATE TABLE ""streets"" (""id"" INTEGER PRIMARY K..."
5,table,garments,garments,73368,"CREATE TABLE ""garments"" (""id"" INTEGER PRIMARY ..."
6,table,tags,tags,83538,"CREATE TABLE ""tags"" (""id"" INTEGER PRIMARY KEY ..."
7,table,trends,trends,84335,"CREATE TABLE ""trends"" (""id"" INTEGER PRIMARY KE..."
8,table,fanships,fanships,86349,"CREATE TABLE ""fanships"" (""id"" INTEGER PRIMARY ..."
9,table,friendships,friendships,93471,"CREATE TABLE ""friendships"" (""id"" INTEGER PRIMA..."


In [17]:
# Preview 10 indices
schema_list.loc[schema_list['type'] == 'index'][:10]

,type,name,tbl_name,rootpage,sql
16,index,index_users_on_street_id,users,392559,"CREATE INDEX ""index_users_on_street_id"" ON ""us..."
17,index,index_posts_on_user_id,posts,392956,"CREATE INDEX ""index_posts_on_user_id"" ON ""post..."
18,index,index_posts_on_type,posts,394661,"CREATE INDEX ""index_posts_on_type"" ON ""posts"" ..."
19,index,index_posts_on_style_id,posts,397378,"CREATE INDEX ""index_posts_on_style_id"" ON ""pos..."
20,index,index_posts_on_occasion_id,posts,398902,"CREATE INDEX ""index_posts_on_occasion_id"" ON ""..."
21,index,index_posts_on_season_id,posts,400450,"CREATE INDEX ""index_posts_on_season_id"" ON ""po..."
22,index,index_garments_on_clothing_id,garments,401805,"CREATE INDEX ""index_garments_on_clothing_id"" O..."
23,index,index_garments_on_color_id,garments,406651,"CREATE INDEX ""index_garments_on_color_id"" ON ""..."
24,index,index_garments_on_brand_id,garments,411314,"CREATE INDEX ""index_garments_on_brand_id"" ON ""..."
25,index,index_garments_on_trend_id,garments,416255,"CREATE INDEX ""index_garments_on_trend_id"" ON ""..."


In [18]:
# Schema only has ables & indices
schema_list.loc[(schema_list['type'] != 'index') & (schema_list['type'] != 'table') ]

,type,name,tbl_name,rootpage,sql


In [19]:
# Now we want to start creating analytics around the tables
table_names_list = list(schema_list.loc[schema_list['type'] == 'table']['tbl_name'])

table_names_list.remove('schema_migrations')
table_names_list.remove('sqlite_sequence')

table_names_list

['users',
 'posts',
 'streets',
 'garments',
 'tags',
 'trends',
 'fanships',
 'friendships',
 'bookmarks',
 'stores',
 'comments',
 'color_taggings',
 'taggings',
 'photos']

In [36]:
# We're going to grab the tablenames and their columns
table_list_dict = {}


#Note: You cannot parameterize tablenames directly using 
# pandas read_sql
# See link: https://stackoverflow.com/questions/5870284/can-i-use-parameters-for-the-table-name-in-sqlite3
# You can however re-build the query string each time & concatenate tablename 

for t_name in table_names_list:
    query = f'SELECT * FROM {t_name} where 1=2'
    table_list_dict[t_name] = pd.read_sql(query, con=db)

In [31]:
tables_columns = {}

for t_name in table_names_list:
    tables_columns[t_name] = pd.DataFrame(table_list_dict[t_name].columns)
    
tables_columns

{'users':                              0
 0                           id
 1                         name
 2                         type
 3                    street_id
 4                        views
 5                  chic_points
 6                       status
 7                  description
 8              bookmarks_count
 9               comments_count
 10               friends_count
 11                  fans_count
 12  favorite_chictopians_count
 13                  created_at
 14                  updated_at,
 'posts':                   0
 0                id
 1      chictopia_id
 2           user_id
 3              type
 4            status
 5             votes
 6             title
 7              date
 8       description
 9          style_id
 10      occasion_id
 11        season_id
 12   comments_count
 13  bookmarks_count
 14       created_at
 15       updated_at,
 'streets':               0
 0            id
 1          name
 2  chictopia_id
 3   users_count,
 'garments':  

<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style='background :red' > Step 4A: 🔬 Deeper EDA & Exploration of Some of the Relationships 📚</span> 

☑️ Explore samples of the tables

☑️ Understand relationships

## <span style='background :orange' > Get a sample of each of the tables </span>


In [37]:
# Gettin schema

# Tables = ['users','posts','streets','garments','tags','trends','fanships','friendships',
# 'bookmarks','stores','comments','color_taggings','taggings','photos']


users_sample = pd.read_sql("""SELECT * 
FROM users limit 100
""",con=db)

users_sample.sample(n=10)

,id,name,type,street_id,views,chic_points,status,description,bookmarks_count,comments_count,friends_count,fans_count,favorite_chictopians_count,created_at,updated_at
24,25,_stylorectic,Styleicon,13.0,25460,15588,2,<p>http://stylorectic.blogspot.com/</p>\n<p></p>,32,674,1425,1092,40,2012-09-07 23:05:44.123676,2012-12-15 15:22:04.157207
61,63,Amandas_Armoire,Stylecouncil,NaN,7188,2712,2,,6,366,249,114,11,2012-09-07 23:05:50.323072,2012-12-15 15:23:36.321442
4,5,lindarrr,Styleicon,5.0,67730,11792,2,"<p><a rel=""nofollow"" href=""http://www.chictopi...",7,353,585,746,24,2012-09-07 23:05:21.815427,2012-12-15 14:59:40.168670
82,86,athena,Stylecouncil,NaN,1967,555,2,<p>the artist formally known as athena can be ...,13,111,26,2,11,2012-09-07 23:05:52.515512,2012-12-15 15:25:13.834617
52,54,Elenita,Styleicon,34.0,39848,13239,2,"<p><a rel=""nofollow"" href=""http://elenita.no"" ...",5,376,771,999,18,2012-09-07 23:05:48.992975,2012-12-15 15:29:35.571852
90,94,Joachim,Stylecouncil,50.0,6776,2558,2,,0,2,240,132,0,2012-09-07 23:05:53.046797,2012-12-15 15:27:16.334566
38,39,highsociety,Stylecouncil,24.0,4343,800,2,<p>I own an online store High Society Clothing...,15,12,82,24,1,2012-09-07 23:05:47.237984,2012-12-15 15:15:58.057538
32,33,bizzy,Stylecouncil,19.0,13729,5966,2,,0,23,361,294,1,2012-09-07 23:05:46.151928,2012-12-15 15:17:04.224043
62,64,allikat,Stylecouncil,39.0,668,381,2,,36,1,59,0,0,2012-09-07 23:05:50.387951,2012-12-15 15:22:27.888225
75,78,stmannequins,Stylecouncil,30.0,1024,430,2,"<p>simple, comfortable but sparkly</p>",3,13,81,5,3,2012-09-07 23:05:51.649718,2012-12-15 15:24:51.402547


In [45]:
# Gettin schema

# Tables = ['users','posts','streets','garments','tags','trends','fanships','friendships',
# 'bookmarks','stores','comments','color_taggings','taggings','photos']


garments_sample = pd.read_sql("""SELECT * 
FROM garments limit 100
""",con=db)

garments_sample.sample(n=10)

,id,clothing_id,color_id,brand_id,trend_id,store_id,post_id
37,38,NaN,NaN,NaN,34.0,None,146
74,75,60.0,9.0,56.0,61.0,None,264
81,82,31.0,12.0,NaN,NaN,None,3
94,95,63.0,10.0,NaN,NaN,None,17
70,71,31.0,NaN,56.0,56.0,None,252
10,11,NaN,NaN,NaN,11.0,None,98
84,85,60.0,10.0,43.0,NaN,None,277
86,87,16.0,10.0,NaN,NaN,None,7
80,81,42.0,38.0,54.0,NaN,None,277
30,31,16.0,28.0,29.0,NaN,None,13


In [47]:
# Gettin schema

# Tables = ['users','posts','streets','garments','tags','trends','fanships','friendships',
# 'bookmarks','stores','comments','color_taggings','taggings','photos']


taggings_sample = pd.read_sql("""SELECT * 
FROM taggings limit 100
""",con=db)

taggings_sample.sample(n=10)

,id,post_id,trend_id,created_at,updated_at
14,15,12,89,2012-09-07 23:13:33.628753,2012-09-07 23:13:33.628753
50,51,128,157,2012-09-07 23:17:01.889801,2012-09-07 23:17:01.889801
91,92,1221,273,2012-09-07 23:23:20.549136,2012-09-07 23:23:20.549136
47,48,18,151,2012-09-07 23:16:52.801671,2012-09-07 23:16:52.801671
6,7,137,80,2012-09-07 23:12:43.688446,2012-09-07 23:12:43.688446
16,17,260,91,2012-09-07 23:13:46.808778,2012-09-07 23:13:46.808778
90,91,1974,287,2012-09-07 23:22:36.345585,2012-09-07 23:22:36.345585
44,45,388,139,2012-09-07 23:16:51.931053,2012-09-07 23:16:51.931053
51,52,128,57,2012-09-07 23:17:02.039877,2012-09-07 23:17:02.039877
25,26,295,92,2012-09-07 23:14:24.048772,2012-09-07 23:14:24.048772


In [48]:
# Gettin schema

# Tables = ['users','posts','streets','garments','tags','trends','fanships','friendships',
# 'bookmarks','stores','comments','color_taggings','taggings','photos']


trends_sample = pd.read_sql("""SELECT * 
FROM trends limit 100
""",con=db)

trends_sample.sample(n=10)

,id,name,chictopia_id,taggings_count,garments_count
10,11,pink tank,54402,0,3
83,84,"silk, tunic",53161,0,1
81,82,buckles,3676,4,53
28,29,h&m bikini,52072,0,1
61,62,leather pants,3770,899,275
78,79,"givenchy, peep toe, cyana trendland",55347,0,1
20,21,skinny,366,254,4224
52,53,my mom's navy skirt,50892,0,1
94,95,classic,221,2522,320
38,39,gold hoops,12299,0,18


In [49]:
# Gettin schema

# Tables = ['users','posts','streets','garments','tags','trends','fanships','friendships',
# 'bookmarks','stores','comments','color_taggings','taggings','photos']


color_taggings_sample = pd.read_sql("""SELECT * 
FROM color_taggings limit 100
""",con=db)

color_taggings_sample.sample(n=10)

,id,post_id,color_id
98,99,12,12
18,19,45,28
70,71,67,34
37,38,17,10
9,10,35,11
25,26,264,34
99,100,12,11
81,82,385,12
75,76,379,10
65,66,711,82



# <span style='background :pink' > ⬆️ LEFT OFF ABOVE ⬆️  </span> 

## <span style='background :orange' > High-Level Describing </span>

In [ ]:
label_descriptions_df.info()

In [ ]:
label_descriptions_df.describe()

## <span style='background :orange' > Examining individual cells & entries </span>

In [ ]:
# This corresponds to the first row & first column (categories)
label_descriptions_df.loc[0][0]

In [ ]:
# This corresponds to the first row & second column (attributes)
label_descriptions_df.loc[0][1]

In [ ]:
# This corresponds to the second row & first column (categories)
label_descriptions_df.loc[1][0]

In [ ]:
# This corresponds to the second row & second column (attributes)
label_descriptions_df.loc[1][1]

In [ ]:
# This corresponds to the 16th row & first column (categories)
label_descriptions_df.loc[15][0]

## <span style='background :orange' > Examining Nulls </span>

⚠️ The creators of the dataset did something interesting -- they stuck two separate dictioanies together in a single JSON file. So while it looks like there are nulls in the categories column past 46 corresponding to attributes, that's not quite accurate. 

🤔 As we need to explore the "nulls" we need to remember they're not "true nulls" but the result of the dataset author's decision to save space & memory by dumping two lists of separate lengths together.

In [ ]:
# Let's locate all rows of categories that don't have attributes

label_desc_isNull_df = label_descriptions_df.loc[pd.isnull(label_descriptions_df['categories'])]
label_desc_isNull_df

In [ ]:
label_desc_isNull_df.head(20)

In [ ]:
#Let's check out some specfic exaples
label_desc_isNull_df.loc[46][1]

In [ ]:
label_desc_isNull_df.loc[62][1]

In [ ]:
label_desc_isNull_df.loc[65][1]

In [ ]:
label_desc_isNull_df.tail(20)

In [ ]:
label_desc_isNull_df.loc[293][1]

In [ ]:
label_desc_isNull_df.loc[274][1]

In [ ]:
label_desc_isNull_df.loc[287][1]

## <span style='background :orange' > Cleaning Up Categories & Attributes Dicts </span>

### <span style='background :yellow' > Cleaning Up Categories </span>

We're going to pull out and split the categories list from the attributes list. We'll also make the columns easier to track by renaming, etc.

In [ ]:
# We'll save off a copy of the dataframe for inspection. 
# It'll be deleted manually later once we're happy with the quality.

categories_notNull_df = label_descriptions_df.loc[pd.notnull(label_descriptions_df['categories'])]
categories_notNull_df = pd.json_normalize(categories_notNull_df['categories'])
categories_notNull_df.to_csv('../../data/interim/categories_notNull_df.csv')
categories_notNull_df

In [ ]:
categories_notNull_df = categories_notNull_df.rename(
                                columns={'id':'id_categories',
                                        'name':'name_categories',
                                        'supercategory':'supercategory_categories',
                                        'level':'level_categories'})
categories_notNull_df = categories_notNull_df.set_index('id_categories')
categories_notNull_df

In [ ]:
#Checking out the distribution of the categories 
categories_notNull_df['supercategory_categories'].value_counts().plot(kind='bar')

### <span style='background :yellow' > Cleaning Up Attributes </span>

Same exact process as we did with categories above. 

In [ ]:
# We'll save off a copy of the dataframe for inspection. 
# It'll be deleted manually later once we're happy with the quality.

attributes_df = pd.json_normalize(label_descriptions_df['attributes'])
attributes_df.to_csv('../../data/interim/attributes_df.csv')
attributes_df

In [ ]:
# Especially for attributes, it's important we set the id_attributes column as the index
# as we intend to use the dataframes for merging later.
# The reason why we'd want to set the index instead of just using the default pandas index?
# You'll see later but essentially the dataset creators, when creating the JSON file,
# skipped numbering the attribute ID's i.e. starting from Attribute ID=281 skip
# ahead by 40+. 
# Lucikly this seems to align with the attribute ID's captured in the Train file.
# Keep going, more explanation below.


attributes_df = attributes_df.rename(columns={'id':'id_attributes',
                                        'name':'name_attributes',
                                        'supercategory':'supercategory_attributes',
                                        'level':'level_attributes'})
attributes_df = attributes_df.set_index('id_attributes')
attributes_df

In [ ]:
attributes_df['supercategory_attributes'].value_counts().plot(kind='bar')

<hr style="border-top: 5px solid black; margin-top: 1px; margin-bottom: 1px"></hr>

# <span style='background :red' > Step 4B: 🔬 Deeper EDA & Exploration of Train 🚂 File </span> 

## <span style='background :orange' > Loading the train.csv file 🚂 </span> 

We can note the following:
1. The same image is repeated multiple times but with a unique:
    * ClassId
    * Encoded Pixels
    * AttributesIds
    
2. Not all ImageIds have corresponding AttributesIds
3. AtributeIds are off, starting at AttributeId = 281, which jumps from the prior entry of AttributeId = 234. We also see that the column AttributeIds in the train data_df include AttrbiuteId values of 300+. As part of futher processing we'd need to visually confirm that the train images with AttributeId's of 235-293 are correctly labeled. 

In [ ]:
# We load one chunk to visually examine (so that we dont overload the notebook memory)
with pd.read_csv("/Volumes/MiniGator/Projects/Datasets/iMaterialist/train.csv",chunksize=100) as reader:
    print(reader.get_chunk(20))

In [ ]:
# Now I'll load the file straight; for bigger files howver, it can make sense
# to still do lazy loading

train_df = pd.read_csv("/Volumes/MiniGator/Projects/Datasets/iMaterialist/train.csv")

## <span style='background :orange' > Examine the Train file 🚂</span> 

In [ ]:
train_df.head(20)

In [ ]:
train_df.info()

## <span style='background :orange' > Get Top-Level Counts of Combined Train 🚊  File & Classes Dict 📚 </span> 

In [ ]:
# In the Train data:
# ClassId corresponds to the category ID
# Each row in train contains a category label & multiple attribute labels

train_df[['ClassId','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['ClassId']).count().merge(categories_notNull_df,how='left',left_on='ClassId',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_categories')

### <span style='background :yellow' > Visualize entire distribution of supercategory categories </span> 

In [ ]:
train_df[['ClassId','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['ClassId']).count().merge(categories_notNull_df,how='left',left_on='ClassId',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('supercategory_categories').plot(kind='barh',y='counts_',use_index=True)

### <span style='background :yellow' > Visualize top 15 categories by supercategory count</span> 

In [ ]:
train_df[['ClassId','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['ClassId']).count().merge(categories_notNull_df,how='left',left_on='ClassId',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('supercategory_categories').iloc[:15].plot(kind='barh',y='counts_',use_index=True)

### <span style='background :yellow' > Visualize entire distribution of fine category (not super category) count </span> 

In [ ]:
train_df[['ClassId','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['ClassId']).count().merge(categories_notNull_df,how='left',left_on='ClassId',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_categories').plot(kind='barh',y='counts_',use_index=True)

### <span style='background :yellow' > Visualize top 15 of fine category (not super category) count </span> 

In [ ]:
train_df[['ClassId','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['ClassId']).count().merge(categories_notNull_df,how='left',left_on='ClassId',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_categories').iloc[:15].plot(kind='barh',y='counts_',use_index=True)

## <span style='background :orange' > Get Top-Level Counts of Combined Train 🚊  File & Attributes Dict 📚 </span> 

### <span style='background :yellow' > Select out columns to start analysis of attributes</span> 

In [ ]:
# Remember: The same image is represented multiple times with a unique ClassId 
# (aka one image can have multiple items of clothing) and each ClassId (item of clothing)
# can have multiple attributes (details like tpye of fabric, buttons, etc)
# This is why the AttributesIds column has a list of values

train_with_attributes = train_df[['ImageId','ClassId','AttributesIds']]
train_with_attributes

### <span style='background :yellow' > Explode out attributes to make a long train_attributes dataframe </span> 

In [ ]:
# Not all images have associated detailed attributes 
# If we just try to explode out the NaN values, we'll get an error
# So we need to convert the AttributesIds into a list of values (str)
# such that we can then encapsulate as a list, use pd.explode, 
# and then recast as int so we can merge on the indices.
# Phew!

train_with_attributes['AttributesIds'] = train_with_attributes['AttributesIds'].replace(np.nan,-1000).astype(str)
train_with_attributes['AttributesIds'] = train_with_attributes['AttributesIds'].apply(lambda x: list(x.split(",")))

train_with_attributes_long = train_with_attributes.explode('AttributesIds')
train_with_attributes_long['AttributesIds'] = train_with_attributes_long['AttributesIds'].astype(int)
train_with_attributes_long

In [ ]:
# Now we have a count of number of times (not images but occurrences) of the attributes
train_with_attributes_long[['AttributesIds','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['AttributesIds']).count().merge(attributes_df,how='left',left_on='AttributesIds',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_attributes')

In [ ]:
# This should be an ugly mess of a chart -- long tail but we also have a bunch of the 
# top-level categories being repeated
train_with_attributes_long[['AttributesIds','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['AttributesIds']).count().merge(attributes_df,how='left',left_on='AttributesIds',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_attributes').plot(kind='barh',y='counts_',use_index=True)

In [ ]:
# Grabbing just the first 15, we see there are a bunch of occurrences of NaN
# This isnt surprising as the dataset creators described how only a subset of the original data
# had additional detailed attributes information.
train_with_attributes_long[['AttributesIds','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['AttributesIds']).count().merge(attributes_df,how='left',left_on='AttributesIds',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_attributes').iloc[:15].plot(kind='barh',y='counts_',use_index=True)

In [ ]:
# Ignoring Nan, checking out the top 30 attributes
train_with_attributes_long[['AttributesIds','ImageId']].rename(columns={'ImageId':'counts_'}).groupby(['AttributesIds']).count().merge(attributes_df,how='left',left_on='AttributesIds',right_index=True).reset_index().sort_values('counts_',ascending=False).set_index('name_attributes').iloc[1:30].plot(kind='barh',y='counts_',use_index=True)

## <span style='background :orange' > Finishing EDA 🔬 by Previewing Some Random Images 📸 </span> 

💡 Ideally, it would be best to do a random selection of the images to preview by grabbing a list of the image_file names, creating a list of random numbers, then picking the image whose index corresponds to the random number element. 

🤔 Given that this is just meant to be a cursory preview, we'll do that in later noteboks for the dataset that ultimately gets selected.

In [ ]:
path_to_dir = '/Volumes/MiniGator/Projects/Datasets/iMaterialist/train'

In [ ]:
listOfImageNames = ['00000663ed1ff0c4e0132b9b9ac53f6e.jpg',
                    '0000fe7c9191fba733c8a69cfaf962b7.jpg',
                    '0002ec21ddb8477e98b2cbb87ea2e269.jpg',
                    '0002f5a0ebc162ecfb73e2c91e3b8f62.jpg',
                    '0004467156e47b0eb6de4aa6479cbd15.jpg',
                    '00048c3a2fb9c29340473c4cfc06424a.jpg',
                    '0006ea84499fd9a06fefbdf47a5eb4c0.jpg'
                   ]

In [ ]:
for imageName in listOfImageNames:
    display(Image(filename=f'{path_to_dir}//{imageName}'))